# When you return

You need to remove upcoming games before you drop 0 reviews - try make something for XBOX!

In [23]:
import sys
import pandas as pd
from sklearn.preprocessing import MultiLabelBinarizer

sys.path.append("/Users/antonis/code/Ant-mel/legendary_game_recs/")
from preprocessing.preprocess_1_cleaning import *
from preprocessing.preprocess_2_features import *

# Exploring the data

Data had to be scraped in small batches else it failed. 

When scraped during the day, batches would only be 10k as it would crash sooner and the person scraping could take brakes inbetween for whatever reason. 

For reference - a 10k batch would take 2.5 hours

In [24]:
set_1 = pd.read_csv('../raw_data/final_scrape_0to10k')
set_2 = pd.read_csv('../raw_data/final_scrape_10kto40k')
set_3 = pd.read_csv('../raw_data/final_scrape_40kto50k')
set_4 = pd.read_csv('../raw_data/final_scrape_50kto60k')
set_5 = pd.read_csv('../raw_data/final_scrape_60kto70k')
set_6 = pd.read_csv('../raw_data/final_scrape_70kto80k')
set_7 = pd.read_csv('../raw_data/final_scrape_80kto90k')
set_8 = pd.read_csv('../raw_data/final_scrape_90kto120k')

In [25]:
data = pd.concat((set_1, set_2, set_3, set_4, set_5, set_6, set_7, set_8))

In [26]:
data.shape

(111951, 27)

In [27]:
data.head()

,title,release_date,plays,playing,backlogs,wishlist,developers,avg_review,genres,platforms,...,ratings_one_five,ratings_two_zero,ratings_two_five,ratings_three_zero,ratings_three_five,ratings_four_zero,ratings_four_five,ratings_five_zero,url,game_id
0,PilotXross,2030-12-20,1,0,2,2,['PilotXross'],0.0,"['Indie', 'Racing', 'Sport']",['SteamVR'],...,0,0,0,0,0,0,0,0,/games/pilotxross/,123757
1,Plague Hunter,2029-02-07,0,0,2,7,[],0.0,"['Adventure', 'Indie']",['Windows PC'],...,0,0,0,0,0,0,0,0,/games/plague-hunter/,106694
2,The God Slayer,2027-12-31,1,0,1,11,['Pathea Games'],0.0,['RPG'],['PlayStation 5'],...,0,0,0,0,0,0,0,0,/games/the-god-slayer/,259874
3,Rise of Rebellion,2025-12-31,0,0,0,2,['KODANSHA'],0.0,['Adventure'],['Windows PC'],...,0,0,0,0,0,0,0,0,/games/rise-of-rebellion/,211568
4,Penny Blood,2025-12-31,1,0,27,124,[],0.0,['RPG'],"['Windows PC', 'PlayStation 5', 'Xbox Series']",...,0,0,0,0,0,0,0,0,/games/penny-blood/,214912


### Dealing with Duplicates

10,000 duplicates!!!

In [28]:
# Copying data before adjusting
working_data = data.copy()

In [29]:
working_data[['title', 'release_date']].duplicated().sum()

859

In [30]:
working_data.drop_duplicates(subset=['title'],inplace=True)

In [31]:
working_data[['title']].duplicated().sum()

0

### Incorrect dtypes

- release_date - Needs to be a datetime, but not important for the model, only important for the front-end
- developers, genres, platforms - change to lists (Currently they are strings that look like lists)
- plays, platying, backlogs, wishlists, total_reviews_total_lists - 'k' needs to be removed to make them all int


In [32]:
# Adjusting 'fake' lists to real lists
string_to_list_colums = ['developers','genres','platforms']
working_data[string_to_list_colums] = make_list_columns_to_lists(working_data, string_to_list_colums)

In [33]:
# Turning numeric object columns into int
numeric_columns = ['plays','playing','backlogs','wishlist','total_reviews','total_lists']

for col in numeric_columns:
    numeric_objects_reformatted(working_data, col)

#### Removing games with release date 0001-01-01

In [34]:
year_1_mask = working_data['release_date'] == '0001-01-01'

In [35]:
games_that_exist = working_data[~year_1_mask]

In [36]:
games_that_exist.shape

(106701, 27)

## Dealing with non-main games

DLC, expansions and mods should not be part of the video game recomendations (but perhaps something we can do in future❓)

In [37]:
# removing main games and dropping categories column 
# as its no longer useful

only_main_games = only_main_games(games_that_exist)

In [38]:
only_main_games.shape

(93321, 26)

### Cleaning our y

avg_reviews is currently our y

In the dataset, 64404 games do not have an avg_review, which means they have been reviewd less than 3 times. 

We are happy to remove these games as we choose not to recommend games that we assume have not been played buy humans.

In [41]:
main_games_dattime = change_to_datetype(only_main_games, 'release_date')

We need to seperate upcoming games so we can make recommend them to users - these would otherwise be removed as they cannot have reviews, and we drop games with a review score of 0

note: Review scores cannot be zero as the lowers review is 0.5

In [48]:
upcoming_mask = main_games_dattime['release_date'] > pd.Timestamp(datetime.date.today())
upcoming_games = main_games_dattime[upcoming_mask]
release_games = main_games_dattime[~upcoming_mask]

In [50]:
games_that_exist_with_reviews = remove_no_reviews(release_games)
games_that_exist_with_reviews.shape

(37153, 26)

### Creating final dataset

In [53]:
final_data = pd.concat((upcoming_games, games_that_exist_with_reviews))